<a href="https://colab.research.google.com/github/t0brig01/CSE696FinalProject/blob/main/MoralNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import concat
from sklearn import preprocessing
print ("TensorFlow version: " + tf.__version__)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

TensorFlow version: 1.15.2


# **Data Loading/Manipulation**

In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/t0brig01/CSE696FinalProject/main/TrolleyData.csv")

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   _id             230475 non-null  object
 1   Scenario        230475 non-null  object
 2   UserID          230475 non-null  object
 3   Outcome         230475 non-null  int64 
 4   Session_id      230475 non-null  object
 5   Scenario_order  230475 non-null  int64 
 6   Template        230475 non-null  object
 7   answerLeft      230475 non-null  bool  
 8   lang            230475 non-null  object
 9   seenOther       230475 non-null  bool  
 10  country_code    229259 non-null  object
 11  country_full    229259 non-null  object
dtypes: bool(2), int64(2), object(8)
memory usage: 18.0+ MB


In [16]:
df.head()

,_id,Scenario,UserID,Outcome,Session_id,Scenario_order,Template,answerLeft,lang,seenOther,country_code,country_full
0,C3RGCjGPftjMKYHy2,Loop,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,1,Desktop,False,de,True,DE,Germany
1,3F9SfDyofskDPna8R,Footbridge,0000dc12_9518522259818270,0,367585191-9.51852225982e+15,2,Desktop,False,de,True,DE,Germany
2,oz9anPbE6e52TKuJy,Switch,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,3,Desktop,False,de,True,DE,Germany
3,dkAhBaLhzi62Pp6Hp,Footbridge,0002ae2d_2286850331484848,0,1654499857-2.28685033148e+15,1,Desktop,False,en,True,GB,United Kingdom
4,2ussgtw7vCZMNjxSJ,Switch,0002ae2d_2286850331484848,1,1654499857-2.28685033148e+15,2,Desktop,False,en,True,GB,United Kingdom


In [17]:
filtered = df.drop(['Session_id','Template','country_full','_id','answerLeft'], axis=1)
filtered.dropna()
filtered.replace('Loop',0,inplace=True)   #0 is loop trolley 
filtered.replace('Footbridge',1,inplace=True) #1 is footbridge trolley
filtered.replace('Switch',2,inplace=True) #2 is standard trolley

lang = dict()
country = dict()
iLang = 0
iCountry = 0
for index, x in filtered.iterrows():
  if x["lang"] not in lang.keys():
    lang[x["lang"]] = iLang
    iLang = iLang + 1
  if x["country_code"] not in country.keys():
    country[x["country_code"]] = iCountry
    iCountry = iCountry + 1
for key, value in country.items():
  filtered.replace(key,int(value),inplace=True)
for key, value in lang.items():
  filtered.replace(key,int(value),inplace=True)

In [18]:
filtered['country_code'] = filtered['country_code'].astype(int)

In [19]:
filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Scenario        230475 non-null  int64 
 1   UserID          230475 non-null  object
 2   Outcome         230475 non-null  int64 
 3   Scenario_order  230475 non-null  int64 
 4   lang            230475 non-null  int64 
 5   seenOther       230475 non-null  bool  
 6   country_code    230475 non-null  int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 10.8+ MB


In [48]:
def create_dataset(X):
  Xs = X.drop(["UserID","Outcome","lang"],axis = 1)
  Ys = X.drop(["Scenario","UserID","Scenario_order","lang","seenOther","country_code"],axis=1)
  encoder = LabelEncoder()
  encoder.fit(Ys)
  encoded_Ys = encoder.transform(Ys)
  return np.array(Xs),np.array(encoded_Ys)

In [51]:
interval = int(len(filtered.index)/3)
x_train, y_train = create_dataset(filtered.head(interval))
x_test, y_test = create_dataset(filtered.tail(interval))
x_val, y_val = create_dataset(filtered.loc[interval:interval*2])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# **Neural Network**

In [58]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense
from keras.optimizers import RMSprop,SGD
from tensorflow import keras
from keras import optimizers
import tensorflow as tf

def create_model():
  model = Sequential()
  model.add(Dense(8, input_dim=4, kernel_initializer='normal',activation='relu'))
  model.add(Dense(4, activation='relu',kernel_initializer='normal'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  # history = model.fit(x_train,y_train,epochs=10, validation_data=(x_val,y_val),batch_size=16)

In [ ]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=10, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(pipeline, x_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [44]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)

In [46]:
print(loss)
print(acc)

0.5792068801646197
0.7015684843063354
